In [ ]:
from src import data
import pandas as pd
import math
import numpy as np
from tqdm import tqdm

In [ ]:
speed = data.speeds('train')
bsm = pd.read_csv('/home/giovanni/Desktop/DMProject/resources/dataset/preprocessed/base_structure_df_weather.csv.gz')
bsm = bsm.drop(['Unnamed: 0'], axis=1)
bsm.DATETIME_UTC = pd.to_datetime(bsm.DATETIME_UTC)
bsm['hour'] = pd.to_datetime(bsm.DATETIME_UTC).dt.hour

In [ ]:
bsm.WEATHER.unique()

In [ ]:
bsm

In [ ]:
d = {}
bsm_dropped_duplicates = bsm.drop_duplicates(subset=['KEY', 'KM'])[['KEY', 'KM']]
keys = bsm_dropped_duplicates.KEY.values
kms = bsm_dropped_duplicates.KM.values
for idx in tqdm(range(len(kms))):
    key = keys[idx]
    km = kms[idx]
    fs = bsm[(bsm.KEY == key) & (bsm.KM == km)]
    speeds = speed[(speed.KEY == key) & (speed.KM == km)]
    fmerged = speeds.merge(fs)
    for h in range(24):
        merged = fmerged[(fmerged.hour == h)]        
        mean_street = merged.SPEED_AVG.mean()
        for wt in merged.WEATHER.unique():
            slicew = merged[merged.WEATHER == wt]
            slice_mean = slicew.SPEED_AVG.mean()
            if type(wt) != float:
                if wt not in d:
                    d[wt] = [mean_street-slice_mean]
                else:
                    d[wt].append(mean_street-slice_mean)
    #print('weather condition {}. mean: {}. diff: {}'.format(wt, slice_mean, abs(mean_street-slice_mean)))
#d

In [ ]:
d_mean = {}
for k,v in d.items():
    d_mean[k] = (-sum(v)/len(v), len(v))
d_mean

In [ ]:
len(d)

In [ ]:
d_mean = {}
for k,v in d.items():
    d_mean[k] = -sum(v)/len(v)
d_mean